<a href="https://colab.research.google.com/github/hurricane195/Intro-to-Machine-Learning/blob/Homework-6/HW6_P1_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problem 1:**

b. We will increase the network complexity by adding two additional hidden layers, the hidden layers overall. My suggestions for the size of layers are: 32, 64, 16, respectively. Please redesign the network and compare your training loss value and validation results against the linear regression you implemented in Homework 5 and Problem 1.a. Can you compare your model complexity? Note: Use the same 20%, and 80% split for training and validation.

In [ ]:
#Using modified Temperature example from 'Deep-Learning-with_PyTorch' in Chapter 6 'Using a neural network to fit the data'.
#Using modified Temperature example from 'Deep-Learning-with_PyTorch' in Chapter 2 'Pretrained Networks'.
#Using modified Temperature example "Deep Learning with PyTorch: Optimizers How our models learn!" at https://www.greghilston.com/post/3_optimizers/
#Using modified examples for loss from "Mastering PyTorch Loss Functions: The Complete How-To" at https://www.projectpro.io/article/pytorch-loss-functions/880
#Using modified examples for loss from "Noetebook Community: Linear Regression" at https://notebook.community/aidiary/notebooks/pytorch/180130-linear-regression
#Using modified examples for loss from "PyTorch Basics: Tensors & Gradients" at https://www.evanmarie.com/content/files/notebooks/deep_learning_zero_to_gans_pdf.pdf

In [ ]:
from matplotlib import pyplot as plt     # Data Visualisation
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#import data from .csv file of guithub repository
housing = pd.read_csv('https://raw.githubusercontent.com/hurricane195/Intro-to-Machine-Learning/Datasets/Housing.csv')
housing

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [ ]:
# List of variables to map (string to numerical values)
varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, "no": 0})

# Applying the function to the housing list
housing[varlist] = housing[varlist].apply(binary_map)

# Datacheck
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [ ]:
varlist2 = ['furnishingstatus']

# Defining the map function
def furnishing_map(a):
    return a.map({'furnished': 2, 'semi-furnished': 1, 'unfurnished': 0})

# Applying the function to the housing list
housing[varlist2] = housing[varlist2].apply(furnishing_map)

# Datacheck
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2


In [ ]:
#selected input variables: ALL
num_vars = ['price', 'area','bedrooms',	'bathrooms', 'stories',	'mainroad',	'guestroom', 'basement', 'hotwaterheating',	'airconditioning', 'parking',	'prefarea',	'furnishingstatus']
new_df = housing[num_vars]
new_df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,1,0,1,0,0,2,0,0
541,1767150,2400,3,1,1,0,0,0,0,0,0,0,1
542,1750000,3620,2,1,1,1,0,0,0,0,0,0,0
543,1750000,2910,3,1,1,0,0,0,0,0,0,0,2


In [ ]:
new_df.shape

(545, 13)

In [ ]:
#Normalization
scaler = MinMaxScaler()
#scaler = StandardScaler()
new_df[num_vars] = scaler.fit_transform(new_df[num_vars])
new_df.head(10)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,1.000000,0.396564,0.6,0.333333,0.666667,1.0,0.0,0.0,0.0,1.0,0.666667,1.0,1.0
1,0.909091,0.502405,0.6,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,1.000000,0.0,1.0
2,0.909091,0.571134,0.4,0.333333,0.333333,1.0,0.0,1.0,0.0,0.0,0.666667,1.0,0.5
3,0.906061,0.402062,0.6,0.333333,0.333333,1.0,0.0,1.0,0.0,1.0,1.000000,1.0,1.0
4,0.836364,0.396564,0.6,0.000000,0.333333,1.0,1.0,1.0,0.0,1.0,0.666667,0.0,1.0
5,0.787879,0.402062,0.4,0.666667,0.000000,1.0,0.0,1.0,0.0,1.0,0.666667,1.0,0.5
6,0.727273,0.476289,0.6,0.666667,1.000000,1.0,0.0,0.0,0.0,1.0,0.666667,1.0,0.5
7,0.727273,1.000000,0.8,0.666667,0.333333,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
8,0.703030,0.443299,0.6,0.000000,0.333333,1.0,1.0,1.0,0.0,1.0,0.666667,1.0,1.0
9,0.696970,0.281787,0.4,0.333333,1.000000,1.0,1.0,0.0,0.0,1.0,0.333333,1.0,0.0


In [ ]:
X = new_df.iloc[:, 1:13].values
Y = new_df.iloc[:, 0].values

In [ ]:
#Tensor Conversion

"""
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)
"""

X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
#80% (training) and 20% (validation) split
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

#shuffling dataset
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [ ]:
train_X = X[train_indices]
train_Y = Y[train_indices]

val_X = X[val_indices]
val_Y = Y[val_indices]

In [ ]:
#Adding an extra dimension
train_Y = train_Y.unsqueeze(1)
val_Y = val_Y.unsqueeze(1)

In [ ]:
print('Train X: ', train_X.shape)
print('Train Y: ', train_Y.shape)
print('Val X: ', val_X.shape)
print('Val Y:', val_Y.shape)

print('')

print('Train X: ', train_X.dtype)
print('Train Y: ', train_Y.dtype)
print('Val X: ', val_X.dtype)
print('Val Y: ', val_Y.dtype)

Train X:  torch.Size([436, 12])
Train Y:  torch.Size([436, 1])
Val X:  torch.Size([109, 12])
Val Y: torch.Size([109, 1])

Train X:  torch.float64
Train Y:  torch.float64
Val X:  torch.float64
Val Y:  torch.float64


In [ ]:
import torch.nn as nn

# Fully connected neural network having 12 input feature to 32 hidden features

seq_model1 = nn.Sequential(
            nn.Linear(12,32),
            nn.Tanh(),
            nn.Linear(32,64),
            nn.Tanh(),
            nn.Linear(64,16))

seq_model1 = seq_model1.float()

In [ ]:
optimizer = optim.Adam(seq_model1.parameters(),lr=1e-1)

In [ ]:
n_epochs=500
loss_fn = nn.MSELoss()

for epoch in range(1, n_epochs + 1):
    train_YP = seq_model1(train_X.float())
    train_loss = loss_fn(train_YP, train_Y.float())

    val_YP = seq_model1(val_X.float())
    val_loss = loss_fn(val_YP, val_Y.float())

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
"""
    if epoch <= 1 or epoch % 50 ==0:
        print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
              f" Validation loss {val_loss.item():.4f}")
"""

'\n    if epoch <= 1 or epoch % 50 ==0:\n        print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"\n              f" Validation loss {val_loss.item():.4f}")\n'

In [ ]:
#Define Training loop
def training_loop(n_epochs, optimizer, model, loss_fn, train_X, val_X, train_Y, val_Y):

    for epoch in range(1, n_epochs + 1):
        train_YP = seq_model1(train_X.float())
        train_loss = loss_fn(train_YP, train_Y.float())

        val_YP = seq_model1(val_X.float())
        val_loss = loss_fn(val_YP, val_Y.float())

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <= 1 or epoch % 50 ==0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")


In [ ]:
training_loop(
    n_epochs = 500,
    optimizer = optimizer,
    model = seq_model1,
    loss_fn = nn.MSELoss(),
    train_X = train_X,
    val_X = val_X,
    train_Y = train_Y,
    val_Y = val_Y)


Epoch 1, Training loss 0.0057, Validation loss 0.0132
Epoch 50, Training loss 0.0067, Validation loss 0.0136
Epoch 100, Training loss 0.0060, Validation loss 0.0137
Epoch 150, Training loss 0.0061, Validation loss 0.0138
Epoch 200, Training loss 0.0072, Validation loss 0.0147
Epoch 250, Training loss 0.0129, Validation loss 0.0213
Epoch 300, Training loss 0.0063, Validation loss 0.0152
Epoch 350, Training loss 0.0106, Validation loss 0.0187
Epoch 400, Training loss 0.0049, Validation loss 0.0144
Epoch 450, Training loss 0.0725, Validation loss 0.0794
Epoch 500, Training loss 0.0050, Validation loss 0.0149
